In [3]:
!date

Sun May 18 10:16:16 PM EDT 2025


In [4]:
pwd

'/projects/sccn/andromeda1/aglinska/BC-ABCD-denoise/Code'

In [5]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import ants
import pickle
from tqdm import tqdm

In [6]:
def correlate_columns(arr1, arr2):
    """
    Computes the Pearson correlation between corresponding columns of two matrices.
    
    Parameters:
    arr1 (np.ndarray): First matrix of shape (370, 1000)
    arr2 (np.ndarray): Second matrix of shape (370, 1000)
    
    Returns:
    np.ndarray: 1D array of correlations for each column (size 1000)
    """
    # Ensure input arrays are numpy arrays
    arr1 = np.asarray(arr1)
    arr2 = np.asarray(arr2)
    
    # Subtract the mean of each column (normalize)
    arr1_centered = arr1 - np.mean(arr1, axis=0)
    arr2_centered = arr2 - np.mean(arr2, axis=0)
    
    # Compute the numerator (covariance)
    numerator = np.sum(arr1_centered * arr2_centered, axis=0)
    
    # Compute the denominator (product of standard deviations)
    denominator = np.sqrt(np.sum(arr1_centered**2, axis=0) * np.sum(arr2_centered**2, axis=0))
    
    # Compute the Pearson correlation for each column
    correlation = numerator / denominator
    
    return correlation

In [7]:
def get_regs(events_fn):
    from nilearn.glm.first_level import make_first_level_design_matrix

    events = pd.read_csv(events_fn,delimiter='\t')

    t_r = 2.0 
    #n_scans = epi.shape[-1]
    n_scans = 156
    frame_times = (np.arange(n_scans) * t_r)

    X1 = make_first_level_design_matrix(frame_times,events,drift_model="polynomial",drift_order=3,hrf_model="SPM") #

    face_reg = X1[['face']].values.sum(axis=1)
    place_reg = X1[['body', 'house', 'object', 'scene', 'scramble']].values.sum(axis=1)
    
    return face_reg,place_reg

events_fn_temp = '../Data/StudyForrest/events/{sub}_ses-localizer_task-objectcategories_run-{r}_events.tsv'
face_reg,place_reg = get_regs(events_fn_temp.format(sub='sub-01',r=1))

In [33]:
def get_contrast_val(Y,return_type='contrast'):
    from nilearn.glm.first_level import make_first_level_design_matrix

    events = pd.read_csv(events_fn_temp.format(sub=subs[s],r=r),delimiter='\t')

    t_r = 2.0 
    n_scans = 156  # the acquisition comprises 128 scans
    frame_times = (np.arange(n_scans) * t_r)

    X1 = make_first_level_design_matrix(frame_times,events,drift_model="polynomial",drift_order=3,hrf_model="SPM") #
    
    Y = (Y-Y.mean(axis=1)[:,np.newaxis])/Y.std(axis=1)[:,np.newaxis] # Z score values
    Y = Y.transpose()
    X = X1.values
    beta = np.linalg.inv(X.T @ X1) @ X1.T @ Y
    beta = beta.T
    beta = beta.values

    #['body', 'face', 'house', 'object', 'scene', 'scramble', 'drift_1','drift_2', 'drift_3', 'constant']
    contrast_vector = np.array([-1,5,-1,-1,-1,-1,0,0,0,0]) # Faces > Rest
    contrast_values = beta @ contrast_vector
    if return_type=='contrast':
        return contrast_values.mean()
    elif return_type=='betas':
        return beta.mean(axis=0)

In [11]:
ffa_mask_temp = '../Data/StudyForrest/ROIs/rFFA_final_mask_{sub}_bin.nii.gz'

In [12]:
subs = ['sub-01','sub-02','sub-03','sub-04','sub-05','sub-09','sub-10','sub-14','sub-15','sub-16','sub-17','sub-18','sub-19','sub-20',]

In [16]:
faceMap = ants.image_read('../Misc/ffa_map_resampled_forrest.nii.gz')
faceMap_mask = faceMap.numpy()>5
faceMap_mask.sum()

953

In [21]:
roi = ants.image_read('../Data/StudyForrest/fmriprep/mask_roi.nii')
roni = ants.image_read('../Data/StudyForrest/fmriprep/mask_roni.nii')

In [40]:
def get_result(fn_temp,subFormat='name'):
    result = np.zeros((14,4,3))
    for s in tqdm(range(14)):
        for r in [1,2,3,4]:
            
            if subFormat=='number':
                subim = ants.image_read(os.path.join(fn_temp.format(s=s,r=r)))
            elif subFormat=='name':
                subim = ants.image_read(os.path.join(fn_temp.format(sub=subs[s],r=r)))
            
            subFFA = ants.image_read(os.path.join(ffa_mask_temp.format(sub=subs[s])))
            valsFFA = subim.numpy()[subFFA.numpy()==1]
            valsFFA = valsFFA[valsFFA.std(axis=1)>1e-3,:]

            corr2reg = np.nanmean(correlate_columns(valsFFA.transpose(),np.array([face_reg for _ in range(valsFFA.shape[0])]).transpose()))
            contrastVal = get_contrast_val(valsFFA,return_type='contrast')

            arr1=subim.numpy()[(roi.numpy()==1) * (faceMap_mask==True)]
            arr2=subim.numpy()[(roi.numpy()==1) * (faceMap_mask==False)]

            arr1=arr1[arr1.std(axis=1)>1e-3]
            arr2=arr2[arr2.std(axis=1)>1e-3]
            r1=correlate_columns(arr1.transpose(),np.array([face_reg for _ in range(arr1.shape[0])]).transpose())
            r2=correlate_columns(arr2.transpose(),np.array([face_reg for _ in range(arr2.shape[0])]).transpose())
            specificity = r1.mean()-r2.mean()

            result[s,r-1,0] = corr2reg
            result[s,r-1,1] = contrastVal
            result[s,r-1,2] = specificity
            
    return result

In [38]:
fn_temp = '../Data/StudyForrest/ensembles_last_CVAE/fixed-seed-00-YuOrig-00-DataloaderFix/signal_S{s}_R{r}_avg.nii.gz'
result_CVAE_Yu = get_result(fn_temp,subFormat='number')

100%|██████████| 14/14 [00:43<00:00,  3.14s/it]


In [41]:
fn_temp = '../Data/StudyForrest/fmriprep/{sub}/ses-localizer/func/{sub}_ses-localizer_task-objectcategories_run-{r}_bold_space-MNI152NLin2009cAsym_preproc.nii.gz'
result_preproc = get_result(fn_temp,subFormat='name')

100%|██████████| 14/14 [00:48<00:00,  3.46s/it]


In [42]:
fn_temp = '../Data/StudyForrest/fmriprep/{sub}/ses-localizer/func/{sub}_ses-localizer_task-objectcategories_run-{r}_bold_space-MNI152NLin2009cAsym_COMPCOR-RONI.nii'
result_compcor = get_result(fn_temp,subFormat='name')

100%|██████████| 14/14 [00:38<00:00,  2.75s/it]


In [47]:
fn_temp = '../Data/StudyForrest/ensembles_last_CVAE/DeepCor-Forrest-face-v5/signal_S{s}_R{r}_avg.nii.gz'
result_CVAE_AA = get_result(fn_temp,subFormat='number')

 14%|█▍        | 2/14 [00:06<00:38,  3.22s/it]/home/aglinska/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:232: RuntimeWarning: overflow encountered in multiply
  x = um.multiply(x, x, out=x)
/home/aglinska/anaconda3/lib/python3.8/site-packages/numpy/core/_methods.py:243: RuntimeWarning: overflow encountered in reduce
  ret = umr_sum(x, axis, dtype, out, keepdims=keepdims, where=where)
<ipython-input-6-9f9a06f806b9>:24: RuntimeWarning: overflow encountered in square
  denominator = np.sqrt(np.sum(arr1_centered**2, axis=0) * np.sum(arr2_centered**2, axis=0))
/home/aglinska/anaconda3/lib/python3.8/site-packages/numpy/core/fromnumeric.py:87: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)
100%|██████████| 14/14 [00:45<00:00,  3.24s/it]


In [52]:
model_names=['conv_denoise',
 'conv_denoise500',
 'conv_denoise_bn',
 'conv_denoise_large',
 'conv_denoise_large500',
 'conv_smooth',
 'conv_smooth-small',
 'conv_weights_unet_denoise',
 'conv_weights_unet_denoise500']

In [54]:
fn_temp = '../Data/StudyForrest/Stefano_adv_outputs/01-signals_averaged/S{s}-R{r}-conv_denoise500-avg.nii.gz'
result_conv_denoise500 = get_result(fn_temp,subFormat='number')

100%|██████████| 14/14 [00:49<00:00,  3.54s/it]


In [56]:
fn_temp = '../Data/StudyForrest/Stefano_adv_outputs/01-signals_averaged/S{s}-R{r}-conv_denoise-avg.nii.gz'
result_conv_denoise = get_result(fn_temp,subFormat='number')

100%|██████████| 14/14 [00:47<00:00,  3.42s/it]


In [61]:
print(result_preproc.mean(axis=0).mean(axis=0)/result_preproc.mean(axis=1).std(axis=0))
print(result_compcor.mean(axis=0).mean(axis=0)/result_preproc.mean(axis=1).std(axis=0))
print(result_conv_denoise.mean(axis=0).mean(axis=0)/result_preproc.mean(axis=1).std(axis=0))
print(result_CVAE_Yu.mean(axis=0).mean(axis=0)/result_preproc.mean(axis=1).std(axis=0))
print(result_CVAE_AA.mean(axis=0).mean(axis=0)/result_preproc.mean(axis=1).std(axis=0))

[2.25618443 1.94781238 2.4126718 ]
[2.57257221 2.15056556 2.70172882]
[3.05785635 2.83259817 4.91430045]
[3.21945079 2.06995668 3.76108619]
[3.75894091 2.71520524 4.11080847]


In [57]:
print(result_preproc.mean(axis=0).mean(axis=0))
print(result_compcor.mean(axis=0).mean(axis=0))
print(result_conv_denoise.mean(axis=0).mean(axis=0))
print(result_CVAE_Yu.mean(axis=0).mean(axis=0))
print(result_CVAE_AA.mean(axis=0).mean(axis=0))

[0.1472324  1.86772992 0.07108703]
[0.16787899 2.06214709 0.07960381]
[0.1995473  2.71613857 0.14479508]
[0.21009251 1.98485236 0.11081675]
[0.24529815 2.60357214 0.12112097]
